## Bechmarking using Average of 8-shot GSM8K with maj@8

### V2 Notebook Implementation

In [ ]:
import torch
import re
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteriaList
from datasets import load_dataset
from collections import Counter

# Stopping criteria class
class SpecificStringStoppingCriteria:
    def __init__(self, tokenizer, stop_strings, prompt_length):
        self.tokenizer = tokenizer
        self.stop_strings = stop_strings
        self.prompt_length = prompt_length

    def __call__(self, input_ids, scores, **kwargs):
        if input_ids.shape[1] <= self.prompt_length:
            return False
        
        # Get the generated text only (exclude the prompt)
        generated_text = self.tokenizer.decode(input_ids[0][self.prompt_length:])
        for stop_string in self.stop_strings:
            if stop_string in generated_text:
                return True
        return False

# Helper functions
def extract_predicted_answer(text):
    """Extract the final numerical answer from the model's output."""
    # Look for "The answer is X" pattern
    match = re.search(r"The answer is\s+(\d+\.?\d*|\d*\.\d+)", text)
    if match:
        answer_str = match.group(1).strip()
        try:
            return float(answer_str)
        except ValueError:
            return None
    
    # Fallback: look for the last number in the text
    # This can help when the model doesn't use the exact "The answer is" phrasing
    numbers = re.findall(r"(\d+\.?\d*|\d*\.\d+)", text)
    if numbers:
        try:
            # Take the last number as a possible answer
            return float(numbers[-1])
        except ValueError:
            return None
            
    return None

def extract_ground_truth(text):
    """Extract the numerical answer from the ground truth answer."""
    match = re.search(r"The answer is\s+(\d+\.?\d*|\d*\.\d+)", text)
    if match:
        answer_str = match.group(1).strip()
        try:
            return float(answer_str)
        except ValueError:
            return None
    return None

# 8-shot prompt
EIGHT_SHOT_PROMPT = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost
23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8.

Q: {question}
A:"""

# Main function to evaluate model
def evaluate_model(model_name='mistralai/Mistral-7B-v0.1', majority_method='none', temp=0.7, n_votes=8, num_examples=None, max_new_tokens=512):
    """
    Evaluate a model on the GSM8K dataset.
    
    Parameters:
    - model_name: The HuggingFace model to use
    - majority_method: 'none' (single sample), 'maj@k' (majority voting with k samples), or 'maj@8' (majority voting with 8 samples)
    - temp: Temperature for sampling (higher = more diverse)
    - n_votes: Number of samples to use (only relevant if majority_method is 'maj@k')
    - num_examples: Number of examples to evaluate (None = all)
    
    Returns:
    - Dictionary with results and accuracy
    """
    # Set random seed
    torch.manual_seed(42)
    
    print('Loading model and tokenizer...')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float16)
    
    print('\nLoading dataset...')
    dataset = load_dataset('gsm8k', "main", split='test')
    if num_examples:
        dataset = dataset.select(range(min(num_examples, len(dataset))))
    datasize = len(dataset)
    print(f'Using {datasize} examples from GSM8K test set')
    
    # Override n_votes if using maj@8
    if majority_method == 'maj@8':
        n_votes = 8
        print(f"Using majority voting with {n_votes} samples per question")
    elif majority_method == 'maj@k':
        print(f"Using majority voting with {n_votes} samples per question")
    else:
        n_votes = 1
        print("Using single sample evaluation")
    
    # Define stopping conditions
    generation_util = ["Q:", "</s>", "<|im_end|>"]
    
    results = []
    for i in tqdm(range(datasize), desc='Evaluating'):
        example = dataset[i]
        input_text = EIGHT_SHOT_PROMPT.format(question=example['question'])
        inputs = tokenizer(input_text, return_tensors='pt').to(model.device)
        ground_truth_answer = extract_ground_truth(example['answer'])
        
        stop_criteria = SpecificStringStoppingCriteria(tokenizer, generation_util, len(input_text))
        stopping_criteria_list = StoppingCriteriaList([stop_criteria])
        
        model_answers = []
        if majority_method in ['maj@k', 'maj@8']:
            for _ in range(n_votes):
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs, 
                        temperature=temp, 
                        max_new_tokens=max_new_tokens, 
                        do_sample=True, 
                        pad_token_id=tokenizer.eos_token_id, 
                        stopping_criteria=stopping_criteria_list,
                        # Add these parameters to improve generation relevance
                        top_p=0.95,           # Use nucleus sampling
                        repetition_penalty=1.1, # Penalize repetitions
                        no_repeat_ngram_size=3 # Prevent repeating of 3-grams
                    )
                # Make sure we get the full input + output text
                full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                
                # Extract just the answer part (after the last "A:")
                parts = full_text.split("A:")
                if len(parts) > 1:
                    # Take only what comes after the final "A:"
                    output_text = parts[-1].strip()
                    # Make sure we don't have another question in the output
                    if "Q:" in output_text:
                        output_text = output_text.split("Q:")[0].strip()
                else:
                    output_text = ""
                
                model_answer = extract_predicted_answer(output_text)
                model_answers.append({'text': output_text, 'numeric': model_answer})
        else:
            with torch.no_grad():
                outputs = model.generate(
                    **inputs, 
                    max_new_tokens=max_new_tokens, 
                    pad_token_id=tokenizer.eos_token_id, 
                    stopping_criteria=stopping_criteria_list,
                    repetition_penalty=1.05,   # Slight penalty to avoid repetitions
                    no_repeat_ngram_size=3     # Prevent repeating of 3-grams
                )
            # Make sure we get the full input + output text
            full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Extract just the answer part (after the last "A:")
            parts = full_text.split("A:")
            if len(parts) > 1:
                output_text = parts[-1].strip()
                # Make sure we don't have another question in the output
                if "Q:" in output_text:
                    output_text = output_text.split("Q:")[0].strip()
            else:
                output_text = ""
                
            model_answer = extract_predicted_answer(output_text)
            model_answers.append({'text': output_text, 'numeric': model_answer})
        
        numeric_answers = [ma['numeric'] for ma in model_answers]
        filtered_answers = [num for num in numeric_answers if num is not None]
        majority_answer = Counter(filtered_answers).most_common(1)[0][0] if filtered_answers else None
        
        correct = (majority_answer == ground_truth_answer) if majority_answer is not None else False
        results.append({
            'question': example['question'],
            'gold_answer_text': example['answer'],
            'model_answers_text': [ma['text'] for ma in model_answers],
            'extracted_model_answers': numeric_answers,
            'extracted_gold_answer': ground_truth_answer,
            'majority_answer': majority_answer,
            'correct': correct
        })
    
    # Calculate accuracy
    correct_count = sum(1 for result in results if result['correct'])
    accuracy = correct_count / len(results)
    print(f"Accuracy: {correct_count} / {len(results)} = {accuracy:.4f}")
    
    return {
        'results': results,
        'accuracy': accuracy,
        'model': model_name,
        'majority_method': majority_method,
        'temperature': temp,
        'n_votes': n_votes,
        'max_new_tokens': max_new_tokens
    }

# Example usage in notebook
# results = evaluate_model(
#     model_name='mistralai/Mistral-7B-v0.1',  
#     majority_method='maj@8',      # Options: 'none', 'maj@k', 'maj@8'
#     temp=0.7,                     # Temperature for generation diversity
#     num_examples=10,              # Limit examples for testing
#     max_new_tokens=1024           # Increase token limit if needed
# )

### V1 Notebook implementation

In [ ]:
import torch
import re
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteriaList
from datasets import load_dataset
from collections import Counter

# Stopping criteria class
class SpecificStringStoppingCriteria:
    def __init__(self, tokenizer, stop_strings, prompt_length):
        self.tokenizer = tokenizer
        self.stop_strings = stop_strings
        self.prompt_length = prompt_length

    def __call__(self, input_ids, scores, **kwargs):
        if input_ids.shape[1] <= self.prompt_length:
            return False
        
        # Get the generated text only (exclude the prompt)
        generated_text = self.tokenizer.decode(input_ids[0][self.prompt_length:])
        for stop_string in self.stop_strings:
            if stop_string in generated_text:
                return True
        return False

# Helper functions
def extract_predicted_answer(text):
    """Extract the final numerical answer from the model's output."""
    match = re.search(r"The answer is\s+(\d+\.?\d*|\d*\.\d+)", text)
    if match:
        answer_str = match.group(1).strip()
        try:
            return float(answer_str)
        except ValueError:
            return None
    return None

def extract_ground_truth(text):
    """Extract the numerical answer from the ground truth answer."""
    match = re.search(r"The answer is\s+(\d+\.?\d*|\d*\.\d+)", text)
    if match:
        answer_str = match.group(1).strip()
        try:
            return float(answer_str)
        except ValueError:
            return None
    return None

# 8-shot prompt
EIGHT_SHOT_PROMPT = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost
23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8.

Q: {question}
A:"""

# Main function to evaluate model
def evaluate_model(model_name='mistralai/Mistral-7B-v0.1', majority_method='none', temp=0.7, n_votes=8, num_examples=None):
    """
    Evaluate a model on the GSM8K dataset.
    
    Parameters:
    - model_name: The HuggingFace model to use
    - majority_method: 'none' (single sample), 'maj@k' (majority voting with k samples), or 'maj@8' (majority voting with 8 samples)
    - temp: Temperature for sampling (higher = more diverse)
    - n_votes: Number of samples to use (only relevant if majority_method is 'maj@k')
    - num_examples: Number of examples to evaluate (None = all)
    
    Returns:
    - Dictionary with results and accuracy
    """
    # Set random seed
    torch.manual_seed(42)
    
    print('Loading model and tokenizer...')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float16)
    
    print('\nLoading dataset...')
    dataset = load_dataset('gsm8k', "main", split='test')
    if num_examples:
        dataset = dataset.select(range(min(num_examples, len(dataset))))
    datasize = len(dataset)
    print(f'Using {datasize} examples from GSM8K test set')
    
    # Override n_votes if using maj@8
    if majority_method == 'maj@8':
        n_votes = 8
        print(f"Using majority voting with {n_votes} samples per question")
    elif majority_method == 'maj@k':
        print(f"Using majority voting with {n_votes} samples per question")
    else:
        n_votes = 1
        print("Using single sample evaluation")
    
    # Define stopping conditions
    generation_util = ["Q:", "</s>", "<|im_end|>"]
    
    results = []
    for i in tqdm(range(datasize), desc='Evaluating'):
        example = dataset[i]
        input_text = EIGHT_SHOT_PROMPT.format(question=example['question'])
        inputs = tokenizer(input_text, return_tensors='pt').to(model.device)
        ground_truth_answer = extract_ground_truth(example['answer'])
        
        stop_criteria = SpecificStringStoppingCriteria(tokenizer, generation_util, len(input_text))
        stopping_criteria_list = StoppingCriteriaList([stop_criteria])
        
        model_answers = []
        if majority_method in ['maj@k', 'maj@8']:
            for _ in range(n_votes):
                with torch.no_grad():
                    outputs = model.generate(**inputs, temperature=temp, max_new_tokens=512, 
                                           do_sample=True, pad_token_id=tokenizer.eos_token_id, 
                                           stopping_criteria=stopping_criteria_list)
                output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                output_text = output_text.split("A:")[-1].strip()
                model_answer = extract_predicted_answer(output_text)
                model_answers.append({'text': output_text, 'numeric': model_answer})
        else:
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=512, 
                                       pad_token_id=tokenizer.eos_token_id, 
                                       stopping_criteria=stopping_criteria_list)
            output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            output_text = output_text.split("A:")[-1].strip()
            model_answer = extract_predicted_answer(output_text)
            model_answers.append({'text': output_text, 'numeric': model_answer})
        
        numeric_answers = [ma['numeric'] for ma in model_answers]
        filtered_answers = [num for num in numeric_answers if num is not None]
        majority_answer = Counter(filtered_answers).most_common(1)[0][0] if filtered_answers else None
        
        correct = (majority_answer == ground_truth_answer) if majority_answer is not None else False
        results.append({
            'question': example['question'],
            'gold_answer_text': example['answer'],
            'model_answers_text': [ma['text'] for ma in model_answers],
            'extracted_model_answers': numeric_answers,
            'extracted_gold_answer': ground_truth_answer,
            'majority_answer': majority_answer,
            'correct': correct
        })
    
    # Calculate accuracy
    correct_count = sum(1 for result in results if result['correct'])
    accuracy = correct_count / len(results)
    print(f"Accuracy: {correct_count} / {len(results)} = {accuracy:.4f}")
    
    return {
        'results': results,
        'accuracy': accuracy,
        'model': model_name,
        'majority_method': majority_method,
        'temperature': temp,
        'n_votes': n_votes
    }

# Example usage in notebook
# results = evaluate_model(
#     model_name='mistralai/Mistral-7B-v0.1',  
#     majority_method='maj@8',  # Options: 'none', 'maj@k', 'maj@8'
#     temp=0.7,
#     num_examples=10  # Limit examples for testing
# )

### .py implementation from GutHub https://github.com/tianlwang/eval_gsm8k/blob/main/eval_gsm8k_few_shot.py

In [ ]:
import torch
import re
import os
import argparse
import random
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    StoppingCriteriaList
)
from utils import (
    SpecificStringStoppingCriteria,
    extract_predicted_answer,
    extract_ground_truth
)
from datasets import load_dataset
from collections import Counter
import json


FEW_SHOT_PROMPT = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8.

Q: {question}
A:"""


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', type=str, default='mistralai/Mistral-7B-v0.1')
    parser.add_argument('--use_majority_vote', action='store_true')
    parser.add_argument("--temp", type=float, default=0)
    parser.add_argument('--n_votes', type=int, default=1)
    args = parser.parse_args()


    random_seed = 42
    torch.manual_seed(random_seed)
    random.seed(random_seed)

    print('Loading model and tokenizer...')
    tokenizer = AutoTokenizer.from_pretrained(args.model)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(args.model, device_map='auto', torch_dtype=torch.float16) 
    
    print('\nLoading dataset...')
    dataset = load_dataset('gsm8k', "main", split='test')
    datasize = len(dataset)
    print('gsm8k test size:', datasize) 

    # Define a stopping condition for generation
    generation_util = [
        "Q:",
        "</s>",
        "<|im_end|>"
    ]

    results = []
    for i in tqdm(range(datasize), desc='Evaluating'):
        example = dataset[i]
        input_text = FEW_SHOT_PROMPT.format(question=example['question'])
        inputs = tokenizer(input_text, return_tensors='pt').to(model.device)
        ground_truth_answer = extract_ground_truth(example['answer'])

        stop_criteria = SpecificStringStoppingCriteria(tokenizer, generation_util, len(input_text))
        stopping_criteria_list = StoppingCriteriaList([stop_criteria])

        model_answers = []
        if args.use_majority_vote:
            for _ in range(args.n_votes):
                with torch.no_grad():
                    outputs = model.generate(**inputs, temperature=args.temp, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id, stopping_criteria=stopping_criteria_list)
                output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                # Extract the final answer from the model's output
                output_text = output_text.split("A:")[-1].strip() 
                model_answer = extract_predicted_answer(output_text)
                model_answers.append({'text': output_text, 'numeric': model_answer})
        else:
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=512, pad_token_id=tokenizer.eos_token_id, stopping_criteria=stopping_criteria_list)
            output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            output_text = output_text.split("A:")[-1].strip() 
            model_answer = extract_predicted_answer(output_text)
            model_answers.append({'text': output_text, 'numeric': model_answer})

        numeric_answers = [ma['numeric'] for ma in model_answers]
        filtered_answers = [num for num in numeric_answers if num is not None]
        majority_answer = Counter(filtered_answers).most_common(1)[0][0] if filtered_answers else None

        correct = (majority_answer == ground_truth_answer) if majority_answer is not None else False
        results.append({
            'question': example['question'],
            'gold_answer_text': example['answer'],
            'model_answers_text': [ma['text'] for ma in model_answers],
            'extracted_model_answers': numeric_answers,
            'extracted_gold_answer': ground_truth_answer,
            'majority_answer': majority_answer,
            'correct': correct
        })
    
    cnt = 0
    for result in results:
        if result['correct']:
            cnt += 1
    total = len(results)
    print(f"Accuracy: {cnt} / {total} = {cnt / total :.4f}")

    results.append({'accuracy': cnt / total})

    os.makedirs('eval_results/few_shot', exist_ok=True)
    
    model_name = args.model.split('/')[-1]
    result_file = f"eval_results/few_shot/{model_name}"
    if args.use_majority_vote:
        result_file += f"_maj1@{args.n_votes}_temp{args.temp}"
    result_file += "_results.json"

    with open(result_file, 'w') as f:
        json.dump(results, f, indent=4)

    print(f"Results saved to {result_file}")
                

if __name__ == '__main__':
    main()